In [1]:
%pwd

'f:\\PROJECTS\\Mechine-Learning-Project-with-ML-flow\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'f:\\PROJECTS\\Mechine-Learning-Project-with-ML-flow'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : Path
    local_data_file : Path
    unzip_dir : Path
    

In [5]:
from winequality.constants import *
from winequality.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir

        )
        return data_ingestion_config
    

In [7]:
import os
import urllib.request as request
import zipfile
from winequality import logger
from winequality.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        
        else:
            logger.info(f"file already exists of size : {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path =self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-26 14:20:39,437: INFO: common:yaml file: config\config.yaml loaded successfully]
[2023-12-26 14:20:39,530: INFO: common:yaml file: params.yaml loaded successfully]
[2023-12-26 14:20:39,549: INFO: common:yaml file: schema.yaml loaded successfully]
[2023-12-26 14:20:39,602: INFO: common:created directory at: artifacts_root]
[2023-12-26 14:20:39,606: INFO: common:created directory at: artifacts/data_ingestion]


ValueError: unknown url type: 'Dataset\\winequalitydata.zip'